##### Imports the necessary modules and sets the system path to locate them.

In [1]:
import sys
sys.path.append("../utils")
from astra_sim import AstraSim, Collective, NetworkBackend

##### Connects the client to the AstraSim gRPC server, initializes the AstraSim SDK, and creates a folder (tagged as specified) containing all configuration details, generated results, and logs.

In [2]:
astra = AstraSim(server_endpoint ="172.17.0.2:8989", tag = "analytical_unaware_trial")

Resetting test directory
Successfully connected to gRPC server at 172.17.0.2:8989


##### Generates workload execution traces for each rank and configures the data size, which is mandatory for AstraSim workload configuration.

In [3]:
astra.configuration.common_config.workload = astra.generate_collective(collective=Collective.ALLREDUCE, coll_size= 1024*1024*1024, npu_range=[0,8])
print(astra.configuration.common_config.workload)


Generated 8 et in /workspaces/astra_sim_service/client-scripts/utils/../trial/analytical_unaware_trial/configuration/workload
workload/analytical_unaware_trial


##### Configure the system configurations

In [4]:
astra.configuration.common_config.system.scheduling_policy = astra.configuration.common_config.system.LIFO
astra.configuration.common_config.system.endpoint_delay = 10
astra.configuration.common_config.system.active_chunks_per_dimension = 1
astra.configuration.common_config.system.all_gather_implementation = [astra.configuration.common_config.system.RING]
astra.configuration.common_config.system.all_to_all_implementation = [astra.configuration.common_config.system.DIRECT]
astra.configuration.common_config.system.all_reduce_implementation = [astra.configuration.common_config.system.RING]
astra.configuration.common_config.system.collective_optimization = astra.configuration.common_config.system.LOCALBWAWARE
astra.configuration.common_config.system.local_mem_bw = 1600
print(astra.configuration.common_config.system)


active_chunks_per_dimension: 1
all_gather_implementation:
- ring
all_reduce_implementation:
- ring
all_to_all_implementation:
- direct
collective_optimization: localBWAware
endpoint_delay: 10
local_mem_bw: 1600
local_reduction_delay: 0
preferred_dataset_splits: 1
reduce_scatter_implementation:
- ring
scheduling_policy: LIFO
trace_enabled: 0



##### Configure the remote memory configuration

In [5]:
astra.configuration.common_config.remote_memory.memory_type = astra.configuration.common_config.remote_memory.NO_MEMORY_EXPANSION
print(astra.configuration.common_config.remote_memory)


memory_type: NO_MEMORY_EXPANSION
remote_mem_bw: 0
remote_mem_latency: 0



##### Configure the Network_backend and topology

In [6]:
# astra.configuration.network_backend.choice = astra.configuration.network_backend.ANALYTICAL_CONGESTION_UNAWARE
astra.configuration.network_backend.analytical_congestion_aware.topology.network.clear()
astra.configuration.network_backend.analytical_congestion_unaware.topology.network.add("fullyconnected", 8, 100, 0.005) # add(type_of_topology, number_of_nodes, bandwidth_in_gbps, latency_in_ns)
print("Network backend set to", astra.configuration.network_backend.choice)
print("network backend choice set to:",astra.configuration.network_backend.analytical_congestion_unaware.topology.choice)

Network backend set to analytical_congestion_unaware
network backend choice set to: network


##### Configure the cmd parameters, non-mandatory parameters

In [7]:
astra.configuration.common_config.cmd_parameters.comm_scale = 1
astra.configuration.common_config.cmd_parameters.rendezvous_protocol = False
astra.configuration.common_config.cmd_parameters.injection_scale = 1


#### Start the simulation by providing the network backend name in uppercase letters.

In [8]:
astra.run_simulation(NetworkBackend.ANALYTICAL_CONGESTION_UNAWARE)

Generating Configuration ZIP
output_path: /workspaces/astra_sim_service/client-scripts/utils/../trial/analytical_unaware_trial/config.zip
folder_path: /workspaces/astra_sim_service/client-scripts/utils/../trial/analytical_unaware_trial/configuration
message: 'Configuration applied successfully. warnings: Unable to generate communicator
  group message from schema - communicator group configuration empty'

message: Simulation started successfully

astra-sim server Status: running
Transferring Files from ASTRA-sim server
All files downloaded Successfully
Simulation completed


##### Download all the configurations as a zip

In [9]:
astra.download_configuration()


Downloaded all configuration in /workspaces/astra_sim_service/client-scripts/utils/../trial/analytical_unaware_trial/server_configuration.zip
